In [21]:
import yaml
def dump_yaml(data, filename):
    Dumper = yaml.SafeDumper
    Dumper.ignore_aliases = lambda self, data: True
    with open(filename, 'w', encoding='utf8') as f:
        yaml.dump(data, f, Dumper = Dumper,
                  default_flow_style=False, allow_unicode=True)


In [354]:
def update_mappings(r):
    to_iso = "../data-raw/afrobarometer_to_iso/r%d.yml" % r
    to_wals = "../data-raw/afrobarometer_to_wals/r%d.yml" % r
    out = "../data-raw/afrobarometer-mappings/r%d.yml" % r
    with open(to_wals, 'r') as f:
        data_wals = yaml.load(f)
    dump_yaml(newdata, out)

Add WALS mappings

Generate paths for each ethnologue language

Generate data with distances between languages in the ethnologue. This distance is directed. It is the distance to the closest shared ancestor on the tree.

In [48]:
import csv

lang_ids = set()
with open('../data/afrobarometer_langs.csv', 'r') as f:
    for row in csv.DictReader(f):
        round_ = int(row['round'])
        lang_id = int(row['value'])
        lang_name = row['name']
        key = (round_, lang_id, lang_name)
        lang_ids.add(key)
        

In [51]:
import yaml
for r in range(1, 7):
    
    with open('../data-raw/afrobarometer-mappings/r%d.yml' % r, 'r') as f:
        data = yaml.load(f)
    newdata = []
    for lang in data:
        key = (r, lang['lang_id'], lang['lang_name'])
        if key in lang_ids:
            newdata.append(lang)
    dump_yaml(data, '../data-raw/afrobarometer-mappings/r%d.yml.new' % r)

In [567]:
import yaml
glottolog_manual = []
for r in range(1, 7):
    filename = "../data-raw/afrobarometer-mappings/r%d.yml" % r
    with open(filename, 'r') as f:
        data = yaml.load(f)
    for row in data:
        if 'glottocode' in row and row['glottocode'] is not None:
            if isinstance(row['glottocode'], str):
                glottocode = [{'glottocode2': row['glottocode']}]
            else:
                for lang, countries in row['glottocode'].items():
                    glottocode = [{'glottocode2': lang, 'iso_alpha2': k} 
                                  for k in countries]
            for g in glottocode:
                for q in row['question']:
                    d = {'round': r,
                         'question': q,
                         'lang_id': int(row['lang_id']),
                         'iso_alpha2': None,
                         **g
                        }
                    glottolog_manual.append(d)

glottolog_manual = pd.DataFrame(glottolog_manual)

# get manual matches
glottolog_manual

,glottocode2,iso_alpha2,lang_id,question,round
0,grus1239,None,71,language,1
1,nort3254,None,86,language,1
2,jara1263,None,178,language,1
3,gbag1256,None,274,q97,2
4,gbag1256,None,277,q83,2
5,gbag1256,None,277,q97,2
6,kale1246,None,380,q110,2
7,kale1246,None,380,q97,2
8,luyi1234,None,384,q110,2
9,luyi1234,None,384,q83,2


In [598]:
import newick
with open("/Users/jrnold/Downloads/tree-glottolog-newick.txt", "r") as f:
    glottolog_tree = newick.load(f)

In [599]:
import re

def parse_node(x):
    node_pattern = """^'?
        (?P<name> .* ) [ ]
        \[ (?P<glottocode> [a-z0-9]{8} ) \]
        (?: \[ (?P<iso_639_3> [a-z]{3} ) \] ) ?
        (?P<language> -l- ) ?
    '?$"""
    m = re.match(node_pattern, x, re.X)
    if m is not None:
        out = m.groupdict()
        out['language'] = out['language'] is not None
        return out

In [600]:
def walk_tree(x):
    node = parse_node(x.name)
    node['children'] = [walk_tree(n) for n in x.descendants]
    return node

glottolog_newtree = [walk_tree(x) for x in glottolog_tree]

In [613]:
import os
try:
    os.makedirs('../external/glottolog/')
except FileExistsError:
    pass

In [612]:
with open('../external/glottolog/tree-glottolog.json', 'w') as f:
    json.dump(glottolog_newtree, f, indent = 1)

In [572]:
import csv

langs = {}
filename = "../data/afrobarometer_to_iso_639_3.csv"
with open(filename, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        key = (row['round'], row['question'], row['lang_id'], row['iso_alpha2'])
        if row['iso_scope'] == 'I':
            iso_code = row['iso_639_3']
            try:
                langs[key].add(iso_code)
            except KeyError:
                langs[key] = {iso_code}


In [573]:
glottolog_auto = []
# Keep track of those already matched
for lang, iso_codes in langs.items():
    glottocode = None
    maxdepth = -1
    for leaf in leaves:
        if leaf['level'] > maxdepth and len(iso_codes) > 0:
            if iso_codes.issubset(leaf['iso_codes']):
                maxdepth = leaf['level']
                glottocode = leaf['glottocode']
    glottolog_auto.append({
        'round': int(lang[0]),
        'question': lang[1],
        'lang_id': int(lang[2]),
        'iso_alpha2': lang[3],
        'glottocode': glottocode
    })
glottolog_auto = pd.DataFrame(glottolog_auto)

In [529]:
import pandas as pd
import numpy as np

In [558]:
glottolog_manual.dtypes

glottocode2    object
iso_alpha2     object
lang_id         int64
round           int64
dtype: object

In [554]:
# remove already matched languages
glottolog_auto.merge(glottolog_manual[('round', 'question', 'lang_id')],
                     on = ('round', 'question', 'lang_id'),
                     how = 'left', indicator = True)

KeyError: ('round', 'question', 'lang_id')

In [530]:
afrobarometer_langs = pd.read_csv("../data/afrobarometer_langs.csv",
               na_values = '', keep_default_na = False).\
    astype({'value': np.int64}).\
    rename(columns = {'value': "lang_id"})

In [531]:
afrobarometer_langs = afrobarometer_langs.merge(glottocode_manual.\
                              rename(columns = {'value': 'lang_id'}),
                          how = 'outer',
                          on = ('round', 'question', 'lang_id', 'country'))


,round,question,lang_id,name,country,iso_alpha2,glottocode
0,1,language,1,Afrikaans,1,BW,afri1274
1,1,language,1,Afrikaans,6,NA,afri1274
2,1,language,1,Afrikaans,8,ZA,afri1274
3,1,language,2,Chewa,1,BW,nyan1308
4,1,language,2,Chewa,4,MW,nyan1308
5,1,language,2,Chewa,12,ZW,nyan1308
6,1,language,3,Chinyungwe,4,MW,nyun1248
7,1,language,3,Chinyungwe,6,NA,nyun1248
8,1,language,4,Chisena,4,MW,mala1475
9,1,language,4,Chisena,11,ZM,mala1475


In [494]:
afrobarometer_langs

,round,question,lang_id,name,country,iso_alpha2
0,1,language,1,Afrikaans,1,BW
1,1,language,1,Afrikaans,6,NA
2,1,language,1,Afrikaans,8,ZA
3,1,language,2,Chewa,1,BW
4,1,language,2,Chewa,4,MW
5,1,language,2,Chewa,12,ZW
6,1,language,3,Chinyungwe,4,MW
7,1,language,3,Chinyungwe,6,NA
8,1,language,4,Chisena,4,MW
9,1,language,4,Chisena,11,ZM
